In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, ConfusionMatrixDisplay
from category_encoders import TargetEncoder
import joblib

from sklearn import set_config
set_config(transform_output="pandas")


> **💡 Pro-Tip de Producción:**
> Desde Scikit-Learn 1.2, podemos usar `set_config(transform_output="pandas")`.
> Esto es un **game changer** para debuggear pipelines, ya que no perdemos los nombres de las columnas al transformar los datos.

## 1. Carga de Datos y Split Inicial
**Regla de Oro:** Dividir antes de tocar nada.

In [ ]:
from recursos.utils import load_data
import sys
import os
sys.path.append(os.path.abspath('../../'))

df = load_data('credit_scoring.csv')

# Ver columnas disponibles
print("Columnas:", df.columns.tolist()[:10], "...")
print(f"Dimensiones: {df.shape}")

# Separar Features y Target
TARGET_COL = 'target_y'
COLS_TO_DROP = [TARGET_COL, 'malo_sf_inicio', 'periodo', 'Unnamed: 0']

# ✅ SOLUCIÓN ANTI-PATTERN 3 (IDs como Features):
# TODO: Elimina las columnas que no aportan valor predictivo
# Pista: Usa df.drop() con una lista de columnas existentes
# X = df.drop([col for col in ... if col in df.columns], axis=1)
# y = df[...]

# ✅ SOLUCIÓN ANTI-PATTERN 1 (Data Leakage):
# TODO: Divide en Train/Test ANTES de calcular cualquier estadística
# Pista: Usa train_test_split con stratify=y para mantener proporciones
# X_train, X_test, y_train, y_test = train_test_split(..., ..., test_size=0.2, stratify=..., random_state=42)

# Descomenta cuando completes:
# print(f"\nTrain shape: {X_train.shape}")
# print(f"Test shape: {X_test.shape}")
# print(f"Distribución target (Train): {y_train.value_counts(normalize=True).to_dict()}")


### ⚠️ Real-World Warning: Validación Temporal (OOT)
En este ejercicio usamos `train_test_split` aleatorio, que es estándar para aprender. Sin embargo, en **Credit Scoring** y fraudes, el tiempo es crítico.
*   **El Problema:** El comportamiento de los clientes cambia con la economía.
*   **En la Vida Real:** Se usa **Out-of-Time (OOT) Validation**.

### 📂 Diccionario de Datos (Credit Scoring)
Estamos trabajando con un dataset real de **Riesgo de Crédito para Empresas**.

**Variables Clave:**
*   `target_y`: **Variable Objetivo**. 1 = Cliente Incumplidor (Bad), 0 = Cliente Cumplidor (Good).
*   `banca`, `sector_final`: Segmentación del cliente (Categóricas).
*   `NumeroTrabajadores`: Tamaño de la empresa.
*   `MAX_PORC_DEUDA...`: Variables de comportamiento financiero.
*   `EF_...`: Variables de Estados Financieros.

## 2. Definición de Selectores
Identificamos qué columnas son numéricas y cuáles categóricas.

In [ ]:
# TODO: Identifica los tipos de columnas automáticamente
# Pista: Usa X_train.select_dtypes(include=[...]).columns.tolist()

# Para numéricas: include=['int64', 'float64']
# num_features = X_train.select_dtypes(include=[...]).columns.tolist()

# Para categóricas: include=['object', 'category']
# cat_features = X_train.select_dtypes(include=[...]).columns.tolist()

# Descomenta cuando completes:
# print(f"Features numéricas ({len(num_features)}): {num_features[:5]}...")
# print(f"Features categóricas ({len(cat_features)}): {cat_features[:5] if cat_features else 'Ninguna'}")

# Verificar valores nulos
# null_pct = (X_train.isnull().sum() / len(X_train) * 100).sort_values(ascending=False)
# print(f"\nColumnas con más nulos:\n{null_pct.head()}")


## 3. Construcción de Pipelines Específicos

### ⚠️ Real-World Warning: No todos los Nulos son iguales
Aquí usaremos `SimpleImputer(strategy='median')` por simplicidad, pero cuidado:
1.  **Nulos Estructurales:** Si `Deuda_Tarjeta` es `NaN`, ¿significa que el cliente **NO tiene tarjeta**?
2.  **Nulos Informativos:** No declarar ingresos puede ser un predictor de riesgo en sí mismo.
*   **Consejo Pro:** Investiga el **origen** del nulo antes de imputar.

In [ ]:
# Pipeline Numérico: Imputación + Escalado
# TODO: Crea un Pipeline con dos pasos:
# 1. 'imputer': SimpleImputer con strategy='median'
# 2. 'scaler': StandardScaler

# num_pipeline = Pipeline([
#     ('imputer', SimpleImputer(strategy=...)),  # ✅ Evita Data Leakage: calcula mediana solo en train
#     ('scaler', ...)  # ✅ Evita problemas de convergencia en modelos lineales
# ])

# Pipeline Categórico
# TODO: Crea un Pipeline con dos pasos:
# 1. 'imputer': SimpleImputer con strategy='constant' y fill_value='missing'
# 2. 'encoder': TargetEncoder con min_samples_leaf=10

# cat_pipeline = Pipeline([
#     ('imputer', SimpleImputer(strategy='constant', fill_value=...)),
#     ('encoder', TargetEncoder(min_samples_leaf=...))  # ✅ Maneja categorías nuevas sin romper
# ])


## 4. El ColumnTransformer (El "Router")
Une los pipelines específicos y los aplica a las columnas correctas.

In [ ]:
# TODO: Crea un ColumnTransformer que aplique:
# - num_pipeline a las columnas en num_features
# - cat_pipeline a las columnas en cat_features

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', num_pipeline, ...),  # Pipeline numérico a features numéricas
#         ('cat', cat_pipeline, ...)   # Pipeline categórico a features categóricas
#     ],
#     verbose_feature_names_out=False,
#     remainder='drop'  # Lo que no listamos, se borra
# )

# Visualizar el preprocesador
# preprocessor


## 5. El Pipeline Final (Preprocesamiento + Modelo)
Usamos `LogisticRegression` con `class_weight='balanced'` para manejar el desbalance de clases.

In [ ]:
# TODO: Crea el Pipeline final que combine:
# 1. 'preprocessor': El ColumnTransformer que creaste arriba
# 2. 'classifier': LogisticRegression con class_weight='balanced', solver='liblinear', random_state=42

# model_pipeline = Pipeline([
#     ('preprocessor', ...),
#     ('classifier', LogisticRegression(
#         class_weight=..., solver='liblinear', random_state=42))
# ])


### ⚠️ Real-World Warning: Probabilidades Calibradas
Al usar `class_weight='balanced'`, las probabilidades del modelo estarán **distorsionadas**.
*   **Solución:** Aplicar **Calibración de Probabilidades** (`CalibratedClassifierCV`) si el negocio necesita probabilidades exactas.

## 6. Entrenamiento y Evaluación

In [ ]:
# TODO: Entrena el pipeline completo
# model_pipeline.fit(..., ...)

# TODO: Genera predicciones sobre X_test (¡NO sobre X_train!)
# ✅ SOLUCIÓN ANTI-PATTERN 5 (Evaluar en Train): Evaluamos en TEST, no en train
# y_pred = model_pipeline.predict(...)
# y_proba = model_pipeline.predict_proba(...)[:, 1]  # Probabilidad de clase 1 (default)

# Métricas (Descomenta cuando completes)
# print(classification_report(y_test, y_pred))
# print(f"ROC-AUC Score: {roc_auc_score(y_test, y_proba):.4f}")


In [ ]:
# Visualización de Resultados (Descomenta cuando el modelo esté entrenado)
# plt.figure(figsize=(8, 6))
# ConfusionMatrixDisplay.from_estimator(
#     model_pipeline, X_test, y_test, cmap='Blues', normalize='true')
# plt.title("Matriz de Confusión Normalizada")
# plt.show()


### ⚠️ Real-World Warning: El Negocio manda (Cost-Sensitive Learning)
Un AUC de 0.85 se ve bien en el paper, pero en el banco lo que importa es el dinero.
*   **Falso Positivo:** Rechazamos a un buen cliente -> Perdemos intereses.
*   **Falso Negativo:** Le prestamos a quien no paga -> Perdemos el capital.
*   **Consejo:** En producción, movemos el umbral de decisión para minimizar el **Costo Esperado**, no para maximizar el Accuracy.

## 7. Serialización (Guardar para Producción)
Guardamos el objeto `model_pipeline` completo.

In [ ]:
# TODO: Guarda el pipeline entrenado usando joblib
# Pista: joblib.dump(objeto, 'nombre_archivo.joblib')

# joblib.dump(..., 'baseline_pipeline.joblib')
# print("Pipeline guardado exitosamente.")


---
## 🏆 Resumen de Logros
¡Felicidades! Has construido un pipeline profesional que:
1.  **Es Reproducible:** Cualquier persona puede ejecutar `model_pipeline.predict(nuevo_dato)`.
2.  **Evita Data Leakage:** El split se hizo al principio y los transformadores se ajustaron solo con `X_train`.
3.  **Maneja Nulos y Tipos:** No importa si llegan nulos en producción, el pipeline sabe qué hacer.
4.  **Es un Baseline Sólido:** Tienes un AUC de referencia para intentar superar.

👉 **Siguiente Paso:** En la próxima sesión veremos **Optimización de Hiperparámetros**.